In [3]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [ ]:
VERSION = 2
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [5]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
    "SGD": tf.keras.optimizers.legacy.SGD(),
    "SGD_Momentum": tf.keras.optimizers.legacy.SGD(momentum=0.9),
    "RMSprop": tf.keras.optimizers.legacy.RMSprop(),
    "AdamW": tf.keras.optimizers.AdamW(),
    "Nadam": tf.keras.optimizers.Nadam(),
}

In [7]:
LOSS = {"BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()}

In [8]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-02-28 19:20:29.498512: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-28 19:20:29.498557: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-28 19:20:29.498676: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-28 19:20:29.498766: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 19:20:29.498788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
BATCH_SIZE = 64

In [ ]:
SUBVERSION = 0
for OPTIMIZER_NAME, _ in OPTIMIZERS.items():
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS[OPTIMIZER_NAME],
        loss=LOSS["BinaryCrossentropy"],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-02-28 19:20:31.378219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - ETA: 0s - loss: 0.2377 - accuracy: 0.8957 - precision: 0.9204 - recall: 0.9401 - auc: 0.9544

2025-02-28 19:21:21.578974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1120/1120 [==============================] - 62s 54ms/step - loss: 0.2377 - accuracy: 0.8957 - precision: 0.9204 - recall: 0.9401 - auc: 0.9544 - val_loss: 0.2000 - val_accuracy: 0.9146 - val_precision: 0.9581 - val_recall: 0.9108 - val_auc: 0.9755
Epoch 2/10
1120/1120 [==============================] - 63s 56ms/step - loss: 0.1911 - accuracy: 0.9193 - precision: 0.9427 - recall: 0.9484 - auc: 0.9711 - val_loss: 0.1797 - val_accuracy: 0.9265 - val_precision: 0.9520 - val_recall: 0.9362 - val_auc: 0.9785
Epoch 3/10
1120/1120 [==============================] - 64s 58ms/step - loss: 0.1779 - accuracy: 0.9252 - precision: 0.9487 - recall: 0.9501 - auc: 0.9748 - val_loss: 0.1721 - val_accuracy: 0.9295 - val_precision: 0.9496 - val_recall: 0.9436 - val_auc: 0.9800
Epoch 4/10
1120/1120 [==============================] - 65s 58ms/step - loss: 0.1710 - accuracy: 0.9289 - precision: 0.9508 - recall: 0.9531 - auc: 0.9768 - val_loss: 0.1855 - val_accuracy: 0.9242 - val_precision: 0.9631 - val_reca

...

140/140 [==============================] - 61s 428ms/step - loss: 0.2688 - accuracy: 0.8906 - precision: 0.9174 - recall: 0.9330 - auc: 0.9518 - val_loss: 0.1969 - val_accuracy: 0.9214 - val_precision: 0.9392 - val_recall: 0.9422 - val_auc: 0.9743
Epoch 2/10
140/140 [==============================] - 58s 414ms/step - loss: 0.1984 - accuracy: 0.9171 - precision: 0.9407 - recall: 0.9476 - auc: 0.9690 - val_loss: 0.1861 - val_accuracy: 0.9218 - val_precision: 0.9317 - val_recall: 0.9517 - val_auc: 0.9768
Epoch 3/10
140/140 [==============================] - 58s 415ms/step - loss: 0.1847 - accuracy: 0.9229 - precision: 0.9462 - recall: 0.9497 - auc: 0.9734 - val_loss: 0.1794 - val_accuracy: 0.9298 - val_precision: 0.9477 - val_recall: 0.9461 - val_auc: 0.9785
Epoch 4/10
140/140 [==============================] - 62s 441ms/step - loss: 0.1773 - accuracy: 0.9270 - precision: 0.9489 - recall: 0.9525 - auc: 0.9755 - val_loss: 0.1861 - val_accuracy: 0.9225 - val_precision: 0.9221 - val_recall: 

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a/v1/v1.3/assets


34/34 [==============================] - 11s 323ms/step - loss: 0.1631 - accuracy: 0.9342 - precision: 0.9594 - recall: 0.9404 - auc: 0.9820


2025-02-28 20:02:59.416041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 11s 316ms/step



